# Pandas

表形式のデータの扱いに特化したライブラリ

In [ ]:
from IPython.display import display
import pandas as pd

## データを読み込む

今回はタイタニックデータセットと呼ばれるデータセットを使う。
これは、タイタニック号の乗客の情報と、生存/死亡の関係がラベル付けされたデータ。

この乗客の情報を元に生存/死亡の予測をし、その精度を競うコンペがKaggleのチュートリアルとして公開されている。
[https://www.kaggle.com/c/titanic]()

In [ ]:
data = pd.read_csv('./titanic_train.csv')
display(data)

- この表形式のデータをDataFrameと呼ぶ。

各列の意味([Kaggleのページ](https://www.kaggle.com/c/titanic/data)より)

|名前|意味|
|---|---|
|PassengerId| 乗客番号。ユニークに振られている。|
|Survived| 生存した場合1が入っている。 |
|Pclass | 部屋の等級 |
|Name| 名前 |
|Sex| 性別 |
|SibSp| 一緒に乗船した兄弟/姉妹/夫/妻の人数 |
|Parch| 一緒に乗船した親/子の人数 |
|Ticket| チケット番号|
|Fare| 乗船料|
|Cabin| 部屋番号|
|Ember| 乗船地 C=Cherbourg, Q=Queenstown, S=Southampton|

- 各サンプルにindexが自動的に振られている
- 今回は **PassengerId** という列が有るので、コレをindexとして利用する

In [ ]:
data.index = data.PassengerId
display(data[0:3])  # 多すぎるので最初の3件だけ表示
print len(data)  # 総サンプル数

## 統計量の算出

### 乗船料の最大値, 最小値, 平均, 標準偏差を出してみる

In [ ]:
print('MAX: %f' % data.Fare.max())
print('MIN: %f' % data.Fare.min())
print('AVE: %f' % data.Fare.mean())
print('STD: %f' % data.Fare.std())

## 条件で絞り込む

### 無賃乗船した人を絞り込む

In [ ]:
data_filtered = data[data.Fare == 0]
display(data_filtered[:3])
print(len(data_filtered))

### 年齢が20歳以下の人だけを絞り込む

In [ ]:
data_filtered = data[data.Age < 20]
display(data_filtered[:3])
print len(data_filtered)

### 年齢が60歳以上かつ生存者だけを絞り込む

In [ ]:
data_filtered = data[(data.Age > 60) & (data.Survived == 1)]
display(data_filtered[:3])
print(len(data_filtered))

## データの編集

年齢層別の生存率を調べてみる。１０歳ずつ年齢層を区切ることにし、新しく AgeRange という列を追加する。

In [ ]:
data['AgeRange'] = data.Age // 10  # "//"は切り捨て付きの除算。あまりは切り捨てられ結果が整数で返る
display(data[:3])

AgeRangeでデータをグルーピングする

In [ ]:
# ここ説明不足
display(data.groupby('AgeRange')[['Survived']].agg(['mean', 'count']))

子供の生存率が高いことがわかる。

これは子供から優先的に避難させられたため。同様の理由で女性の生存率も高い?

In [ ]:
display(data.groupby('Sex')[['Survived']].agg(['mean', 'count']))

## データの正規化

### 欠損値の削除

- データには欠損値がつきものなので、コレを前処理の段階で取り除くなどする必要がある。

各列について、欠損がいくつ含まれているか調べる

<img src="figure/nan_0.png", width=600>

In [ ]:
# isnull(): 値が欠損値（NaN）の場合、true (1) を返す。

result = data.isnull().sum(axis=0)

display(result.to_frame())  # 表形式で綺麗に表示するためのもの。

欠損を1つ以上含むサンプルの数を調べる

<img src="figure/nan_1.png", width=600>

In [ ]:
# any(): 行、または列に1つ以上、値が含まれている場合 true (1) を返す。

print(data.isnull().any(axis=1).sum())

全サンプル891件中708件について欠損が含まれているため、欠損を含むサンプルを捨ててしまうと、ほとんどすべてのサンプルが捨てられてしまう。

今回はCabinの列は大半が欠損値なので、列ごと削除してしまう。

AgeRangeの列もとりあえず削除。

In [ ]:
del(data['Cabin'])
del(data['AgeRange'])
display(data[:3])

Ageについては、とりあえず **-1** をセットしておく

In [ ]:
# すでに存在する列のデータを書き換える場合は loc[row, col] を使う
data.loc[data.Age.isnull(), 'Age'] = -1

Embarkedについては欠損値を含むサンプルが少ないので、欠損値を含むサンプルを捨ててしまう。

In [ ]:
data = data.dropna(axis=0)

全ての欠損が消えていることを確認

In [ ]:
result = data.isnull().sum(axis=0)

display(result.to_frame())  # 表形式で綺麗に表示するためのもの。

## numpyの行列に変換する

統計モデルで扱うために、numpyの行列に変換する

In [ ]:
# 数値の列だけ取り出す。
mat = data[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch' , 'Fare']].as_matrix()

print(mat)